## Práctica 2 Búsqueda. Variantes de búsqueda en profundidad y control de estados repetidos.

### Formalización del problema

Importamos en primer lugar las librerías necesarias

In [1]:
import numpy as np
from dataclasses import dataclass
from copy import deepcopy
from __future__ import annotations

Definimos cuáles son los operadores de nuestro problema, para ello hay que tener en cuenta el problema del 8-Puzle:

![](https://upload.wikimedia.org/wikipedia/commons/thumb/4/4e/8puzzle_example.svg/512px-8puzzle_example.svg.png?20110321215916)

*Imagen extraída de https://upload.wikimedia.org/wikipedia/commons/thumb/4/4e/8puzzle_example.svg*

Para ello, usaremos un diccionario o map, en el que la clave, se corresponde con la entrada del usuario para que se realice el operador correspondiente.

In [2]:
operadores = {
    "8": "ARRIBA", 
    "2": "ABAJO", 
    "4": "IZQUIERDA", 
    "6": "DERECHA"
}

Tras esto, podemos definir qué necesitamos para representar el problema. Esta información quedará recogida en un `dataclass`, que es una versión más simple de una clase estándar en Python. Al tener un tablero podemos hacer uso de una matriz para representarlo. Para evitar de una lista de listas, hacemos uso de la librería **Numpy**, que permite tratar la variable como si de una matriz se tratase. Por último, resulta de utilidad tener el control en todo momento de la ubicación del hueco, ya que al fin y al cabo, es donde se va a producir el movimiento. Ésto puede observarse en la siguiente animación:

![](https://camo.githubusercontent.com/8e0521bfd0f2f964d4968a529b134fad5fc1db12fc25c7a8d98428ae88b55cb0/687474703a2f2f6b617869313939332e6769746875622e696f2f696d616765732f70726f6a656374732f3870757a7a6c652f67616d652e676966)

*Animación extraída de https://github.com/kaxi1993/8puzzle*

In [3]:
@dataclass
class tEstado:
    tablero: np.ndarray
    fila: int
    col: int
    N: int
    
    def __init__(self, tablero: np.ndarray):
        self.tablero = tablero
        self.N = self.tablero.shape[0]
        self.fila, self.col = np.where(self.tablero == 0)

    def __repr__(self) -> str:
        return f"{self.tablero}\n Fila: {self.fila}\n Col: {self.col}\n"

    def crearHash(self) -> str:
        return f"{self.tablero.tobytes()}{self.fila}{self.col}"

En `__init__` se tiene el constructor de tEstado. En caso de no estar familiarizado con los conceptos de Orientación a Objetos, piense en tEstado como si fuese una estructura en C. El acceso a los miembros de tEstado es igual.

```python 
tablero = np.array([[0, 2, 3], [1, 4, 5], [8, 7, 6]])
variable = tEstado(tablero) # Suponga tablero una matriz de numpy previamente definida
print(variable.fila, variable.col) # Imprime 0, 0, ya que el hueco está en dicha posición
```

Una vez definida la dataclass, es necesario crear ciertas funciones para poder interactuar. En concreto:

* esValido(op, estado)
* aplicaOperador(op, estado)
* testObjetivo(estado)

Tenga en cuenta que es posible que necesite alguna otra función auxiliar, como `estadoInicial()`, `estadoObjetivo()` o `iguales(estado1, estado2)`. Definamos en primer lugar **esValido**. Se puede apreciar cómo se usa para la estructura `match`, el equivalente a `switch`, `operadores[op]`. Esto se hace para obtener los valores del map, en lugar de trabajar con la clave que es `op`.

In [4]:
def esValido(op: str, estado) -> bool:
    valido = False
    match operadores[op]:
        case "ARRIBA":
            valido = estado.fila > 0
        case "ABAJO":
            valido = estado.fila < estado.N - 1
        case "IZQUIERDA":
            valido = estado.col > 0
        case "DERECHA":
            valido = estado.col < estado.N - 1

    return valido

Continuamos con el código de aplicaOperador. Recordemos que es necesario hacer **explícita** la copia del estado antiguo para poder modificarlo. Para ello, usamos la función `deepcopy` contenida en la librería `copy`.

In [5]:
def aplicaOperador(op, estado):
    nuevo = deepcopy(estado)
    ficha = 0
    match operadores[op]:
        case "ARRIBA":
            ficha = nuevo.tablero[nuevo.fila - 1, nuevo.col]
        case "ABAJO":
            ficha = nuevo.tablero[nuevo.fila + 1, nuevo.col]
        case "IZQUIERDA":
            ficha = nuevo.tablero[nuevo.fila, nuevo.col - 1]
        case "DERECHA":
            ficha = nuevo.tablero[nuevo.fila, nuevo.col + 1]
            
    nuevo.fila, nuevo.col = np.where(nuevo.tablero == ficha)
    nuevo.tablero[nuevo.fila, nuevo.col] = 0
    nuevo.tablero[estado.fila, estado.col] = ficha
    return nuevo

Y por último, la función testObjetivo. Hay que tener en cuenta que aquí sí que nos podría llegar a interesar una función auxiliar que nos compruebe si dos estados son iguales o no. Para ello, podemos comparar directamente los tableros y hacer uso de las ventajas de Numpy. Veamos un ejemplo de esto último antes de ver la implementación de testObjetivo.

In [6]:
puzle_inicial = np.array([[0, 2, 3], [1, 4, 5], [8, 7, 6]])
puzle_final = np.array([[1, 2, 3], [8, 0, 4], [7, 6, 5]])
print(puzle_inicial)
print(puzle_final)

[[0 2 3]
 [1 4 5]
 [8 7 6]]
[[1 2 3]
 [8 0 4]
 [7 6 5]]


In [7]:
puzle_inicial == puzle_final

array([[False,  True,  True],
       [False, False, False],
       [False, False, False]])

Como se puede comprobar, al utilizar el operador ==, se comprueba 1:1 cada elemento de la matriz, obtiendo una matriz de booleanos. Ahora bien, como en este caso se necesita comprobar que **todas** las posiciones sean iguales podemos hacer uso de la operación `all()` (equivale a $\forall$). Si necesitásemos comprobar por el contrario, si **alguna** fuese igual, podríamos hacer uso de la operación `any()` (equivale a $\exists$).

In [8]:
print((puzle_inicial==puzle_final).all())
print((puzle_inicial==puzle_final).any())

False
True


In [9]:
def coste(op, estado):
    return 1 

def estadoInicial() -> tEstado:
    puzle_inicial = np.array([[0, 2, 3], [1, 4, 5], [8, 7, 6]])
    return tEstado(puzle_inicial)

def estadoObjetivo() -> tEstado:
    puzle_final = np.array([[1, 2, 3], [8, 0, 4], [7, 6, 5]])
    return tEstado(puzle_final)

def iguales(actual, objetivo):
    return np.array_equal(actual.tablero, objetivo.tablero)

def testObjetivo(actual):
    objetivo = estadoObjetivo()
    return iguales(actual, objetivo)


Probemos si todo funciona de forma correcta.

In [10]:
estado_ini = tEstado(puzle_inicial)
print(estado_ini)

[[0 2 3]
 [1 4 5]
 [8 7 6]]
 Fila: [0]
 Col: [0]



In [11]:
nuevo_estado = deepcopy(estado_ini)
if esValido("8", estado_ini):
    print("Es posible hacer el movimiento")
else:
    print("No se puede hacer el movimiento\nIntentemos hacer el movimiento hacia Abajo.")
    if esValido("2", estado_ini):
        print("Ahora sí")
        nuevo_estado = aplicaOperador("2", estado_ini)
print(f"El antiguo estado era: \n{estado_ini}")
print(f"El nuevo estado es: \n{nuevo_estado}")

No se puede hacer el movimiento
Intentemos hacer el movimiento hacia Abajo.
Ahora sí
El antiguo estado era: 
[[0 2 3]
 [1 4 5]
 [8 7 6]]
 Fila: [0]
 Col: [0]

El nuevo estado es: 
[[1 2 3]
 [0 4 5]
 [8 7 6]]
 Fila: [1]
 Col: [0]



Parece que todo funciona correctamente, ¿verdad? Pasemos a la búsqueda.

### Búsqueda No Informada

En la búsqueda crearemos una nueva `dataclass` para encapsular el tEstado creado previamente junto con información adicional que nos permita modelar un árbol. 

<img src="https://drive.google.com/uc?id=1MnAr7Fx2fnw_GlaQva1mUsIfTCLb558g" alt="Árbol de busqueda" width="400"/>

Veamos qué información tiene.

In [12]:
@dataclass
class Nodo:
    estado: tEstado
    operador: str
    costeCamino: int
    profundidad: int
    valHeuristica: int  # Por el momento se le puede asignar el valor 0.
    padre: Nodo

    def __str__(self) -> str:
        return f'{"- " * 10}\n{self.estado.tablero}, Operador: {operadores[self.operador]}, Heu:{self.valHeuristica}\n{"- " * 10}'

    def hash(self) -> str:
        return self.estado.crearHash()

Antes de comenzar a describir los atributos, ¿aprecias alguna diferencia con respecto a la dataclass anterior? Dicho de otro modo, ¿echas en falta algún método? ¡Efectivamente! Si simplemente se va a realizar una copia de cada uno de los atributos, no hace falta que creemos de forma explícita la función `__init__()`, si no que por defecto, la tenemos. 

Dicho esto, podemos observar el atributo `estado`, que equivale a lo que hemos visto antes. También tenemos el `operador` que nos llevó a ese estado. Por otro lado, el atributo `padre`, nos indica qué nodo fue el previo. Esto resulta muy útil para poder obtener el camino necesario para ir desde el nodo raíz(que contiene el estado inicial) y uno en el que se encuentra la solución, tal y como se aprecia en la figura anterior. El resto de atributos de `costeCamino`,  `profundidad` y `valHeuristica` nos permitirán variar la búsqueda en función de algunos criterios que se definirán más adelante. Para esta sesión, sí que nos será de utilidad el atributo de `profundidad`, ya que nos indica en qué nivel se encuentra un nodo dado.

In [13]:
def nodoInicial() -> Nodo:
    return Nodo(estadoInicial(), "0", 0, 0, 0, None)

def dispCamino(nodo):
    lista = []
    aux = nodo
    while aux.padre != None:
        lista.append((aux.estado.tablero, aux.operador))
        aux = aux.padre
    for i in lista[::-1]:
        print("Movimiento hacia: ", operadores[i[1]], "\n", i[0])
        print()

def dispSolucion(nodo):
    dispCamino(nodo)
    print("Profundidad: ", nodo.profundidad)
    print("Coste: ", nodo.costeCamino)

Pasemos a la función `expandir()`. Recordemos que para obtener el árbol completo desde un nodo raíz es necesario ir generando sucesores en cada uno de los nodos, para así, poder llegar a la solución. La función sucesores devolverá, dado un nodo, todos los posibles sucesores. ¿En función de qué? Pues de los operadores que se definan para el problema en cuestión. La idea es que sólo podemos aplicar un operador si previamente éste es válido, ¿verdad? Pues este principio es el que contemplamos en la función expandir.

In [14]:
def expandir(nodo) -> list:
    nodos = []

    for op in operadores:
        if esValido(op, nodo.estado):
            nuevo = aplicaOperador(op, nodo.estado)
            nodos.append(
                Nodo(
                    nuevo,
                    op,
                    nodo.costeCamino + coste(op, nuevo),
                    nodo.profundidad + 1,
                    0,
                    nodo,
                )
            )
    return nodos

Puede que no esté familiarizado con los `for` de rango tan habituales de Python. Si es así, veamos un ejemplo antes de continuar. Si definimos una lista con los 10 primeros números naturales, ¿cómo imprimo cada uno de ellos por separado?

In [15]:
naturales = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
for numero in naturales:
    print(numero)

1
2
3
4
5
6
7
8
9
10


In [16]:
# Si en lugar de usar una lista quisiéramos simplemente imprimirlos, lo haríamos como se haría en `C/C++`

for numero in range(1, 11): # Importante, el rango es [1, 11), nótese el intervalo abierto a la derecha
    print(numero)

1
2
3
4
5
6
7
8
9
10


¡Volvamos a la búsqueda en anchura! Si seguimos el método visto en clases de seminarios, podríamos obtener el siguiente pseudocódigo.

```
Solucion: función Búsqueda (tNodo: Inicial)
inicio
    tNodo Actual
    tLista: Abiertos <- {Inicial} // El nodo inicial se guarda en Abiertos
    logico Objetivo: Falso
    mientras (No Vacia(Abiertos)) Y (No Objetivo)
        Actual <- Primero(Abiertos) // Extrae y elimina el primer nodo de Abiertos
        Objetivo <- EsObjetivo(Actual)
        si NO (Objetivo) entonces
            Sucesores <- Expandir(Actual) //calcula heurística a cada sucesor
            Abiertos <- {Abiertos + Sucesores}
        fin_si
        Cerrados <- {Cerrados+Actual}
    fin_mientras
    si Objetivo entonces
        devolver Camino a la Solución 
    si_no 
        devolver Fallo
fin_función
```

In [17]:
def busquedaAnchura() -> bool:
    objetivo = False
    raiz = nodoInicial()
    abiertos = []
    sucesores = []
    cerrados = {}  # Cerrados es un diccionario para que funcione como una tabla hash
    abiertos.append(raiz)

    # Completar el resto del código

    if objetivo:
        dispSolucion()  # Completar
    elif not objetivo:
        if abiertos[0] == raiz:
            print("¡Aún tienes que completar esta función!")
        else:
            print("No se ha encontrado solución")

    return objetivo

# Desplegar sólo si se ha intentado completar el código de la búsqueda en anchura.
def busquedaAnchura_sol() -> bool:
    objetivo = False
    raiz = nodoInicial()
    abiertos = []
    sucesores = []
    cerrados = {}  # Cerrados es un diccionario para que funcione como una tabla hash
    abiertos.append(raiz)

    while not objetivo and len(abiertos)>0:
        actual = abiertos[0]
        abiertos.pop(0)
        objetivo = testObjetivo(actual.estado)
        if not objetivo:
            sucesores = expandir(actual)
            abiertos = abiertos + sucesores
        cerrados[actual.hash()] = 0

    if objetivo:
        dispSolucion(actual)  # Completar
    elif not objetivo:
        print("No se ha encontrado solución")

    return objetivo


In [18]:
objetivo = busquedaAnchura()

¡Aún tienes que completar esta función!


¿Cómo se implementaría la función de la búsqueda en profundidad?

In [19]:
def busquedaProfundidad() -> bool:
    pass # Quite esta línea antes de comenzar a implementar la función

Una vez probado el código de la búsqueda en profundidad. ¿Qué se puede apreciar?



---

¿Cuáles son los próximos pasos? Siga el enunciado del guión de la práctica.

A partir de las implementaciones realizadas en la práctica anterior para el problema del 8-puzle, 
añade las funciones necesarias para obtener las siguientes estrategias de búsqueda: 

1. Búsqueda en Anchura con Control de Estados Repetidos 
2. Búsqueda en Profundidad con Control de Estados Repetidos 
3. Búsqueda con Profundidad Limitada, donde el límite sea un parámetro de entrada de la 
función 
4. Búsqueda con Profundidad Limitada Iterativa

In [20]:
# Implemente aquí el código de la búsqueda en anchura con control de estados repetidos.

In [21]:
# Implemente aquí el código de la búsqueda en profundidad con control de estados repetidos.

In [22]:
# Implemente aquí el código de la búsqueda limitada.

In [23]:
# Implemente aquí el código de la búsqueda limitada iterativa.